In [17]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys, re
import random
from collections import Counter

In [18]:
base_dir = '/home/ubuntu/'
filename = "all_lyrics_no-umlaut.txt"


path = base_dir + filename

try: 
    text = open(path).read().lower()
except UnicodeDecodeError:
    import codecs
    text = codecs.open(path, errors='ignore').read().lower()



In [19]:

text = re.sub('\".*?\"', '', text)  # remove titles of songs
text = text.replace('\n\n\n', '\n')
text = text.replace('\n\n', '\n')
text = text.replace('\r\n', '\n')
    
text = text.replace('  ', ' ')


In [20]:

print('corpus length:', len(text))

chars = set(text)
words = set(text.split())

print("chars:",type(chars))
print("words",type(words))
print("total number of unique words",len(words))
print("total number of unique chars", len(chars))


word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

print("word_indices", type(word_indices), "length:",len(word_indices) )
print("indices_words", type(indices_word), "length", len(indices_word))

maxlen = 30
step = 3
print("maxlen:",maxlen,"step:", step)
sentences = []
next_words = []
list_words = []

sentences2=[]
list_words=text.lower().split()

#split the text into sentences of length maxlen --> sentences 
#split the list of words into list of "next_words" 
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
    
    
print('nb sequences(length of sentences):', len(sentences))
print("length of next_word",len(next_words))



('corpus length:', 144708)
('chars:', <type 'set'>)
('words', <type 'set'>)
('total number of unique words', 3376)
('total number of unique chars', 60)
('word_indices', <type 'dict'>, 'length:', 3376)
('indices_words', <type 'dict'>, 'length', 3376)
('maxlen:', 30, 'step:', 3)
('nb sequences(length of sentences):', 9616)
('length of next_word', 9616)


In [21]:

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1


#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')



Vectorization...
Build model...


In [22]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]



#model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list)



Epoch 1/200


InternalError: Dst tensor is not initialized.
	 [[Node: Const_60 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3376,2048] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'Const_60', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/ubuntu/.local/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/u